# Develop the final state class
## There are 3 parts of the class the user is primarily responsible for
1. Calculations specific to each topology e.g. start time, missing mass,....
2. Calculations common to all topologies, typically the Kinematics of the reaction
3. The Data to be saved in an output tree

## Topology Specific
This requires defining the Topo_X functions where C is the number of the topology.
These are saved in a seperate file ToposXXX.h. We will edit this directly in this Jupyter notebook.

Here you can set the reaction start time to allow calculation of DeltaTime for PID purposes. It may be you want to take this from one particular type of particle such as scattered electron or tagger photon.
You should also calculate the missing mass for this topology and other masses specific to a topology.

Again you will need to change XXX to your class name

In [1]:
//gROOT->ProcessLine(".x $HSCODE/hsfinalstate/LoadFinalState.C+");

%edit ToposK2.h

In [2]:
%%file ToposK2.h
#include "K2.h"
void K2::Topo_0(){
  //For topology Beam:Proton:Kp:Km

  //Subtract starttime from all particles
  //This has been predetermined from the tagger time
  fTrigger.SubtractStartTime(&fProton,&fKp,&fKm);
  fTrigger.SubtractStartTimeBeam(&fProton,&fBeam);

  fProton.TakeCorrectedP();//Apply eloss
  fKp.TakeCorrectedP();//Apply eloss
  fKm.TakeCorrectedP();//Apply eloss

  //Reconstruct missing or combined particles
  HSLorentzVector miss= fBeam.P4() + fTarget -fProton.P4() -fKp.P4() -fKm.P4();
  TD.MissMass2=miss.M2();
  TD.MissMass=miss.M();
  
    //particle kine
  TD.ProtP=fProton.P4p()->P();
  TD.ProtTh=fProton.P4p()->Theta()*TMath::RadToDeg();
  TD.ProtTime=fProton.DeltaTime();
  
  TD.KpP=fKp.P4p()->P();
  TD.KpTh=fKp.P4p()->Theta()*TMath::RadToDeg();
  TD.KpTime=fKp.DeltaTime();
  
  TD.KmP=fKm.P4p()->P();
  TD.KmTh=fKm.P4p()->Theta()*TMath::RadToDeg();
  TD.KmTime=fKm.DeltaTime();
  
  TD.BeamTime=fBeam.DeltaTime();

}
void K2::Topo_1(){
  //For topology Beam:Proton:Kp

  //Subtract starttime from all particles
  //This has been predetermined from the tagger time
  fTrigger.SubtractStartTime(&fProton,&fKp);
  fTrigger.SubtractStartTimeBeam(&fProton,&fBeam);

  fProton.TakeCorrectedP();//Apply eloss
  fKp.TakeCorrectedP();//Apply eloss
  
  //Reconstruct missing or combined particles
  HSLorentzVector miss= fBeam.P4() + fTarget -fProton.P4() -fKp.P4();
  TD.MissMass2=miss.M2();
  TD.MissMass=miss.M();
  fKm.FixP4(miss);
  
    //particle kine
  TD.ProtP=fProton.P4p()->P();
  TD.ProtTh=fProton.P4p()->Theta()*TMath::RadToDeg();
  TD.ProtTime=fProton.DeltaTime();
  
  TD.KpP=fKp.P4p()->P();
  TD.KpTh=fKp.P4p()->Theta()*TMath::RadToDeg();
  TD.KpTime=fKp.DeltaTime();
  
  TD.KmP=0;
  TD.KmTh=0;
  TD.KmTime=0;
  
  TD.BeamTime=fBeam.DeltaTime();

}
void K2::Topo_2(){
  //For topology Beam:Proton:Km

  //Subtract starttime from all particles
  //This has been predetermined from the tagger time
  fTrigger.SubtractStartTime(&fProton,&fKm);
  fTrigger.SubtractStartTimeBeam(&fProton,&fBeam);

  fProton.TakeCorrectedP();//Apply eloss
  fKm.TakeCorrectedP();//Apply eloss

  //Reconstruct missing or combined particles
  HSLorentzVector miss= fBeam.P4() + fTarget -fProton.P4() -fKm.P4();
  TD.MissMass2=miss.M2();
  TD.MissMass=miss.M();
  fKp.FixP4(miss);

  //particle kine
  TD.ProtP=fProton.P4p()->P();
  TD.ProtTh=fProton.P4p()->Theta()*TMath::RadToDeg();
  TD.ProtTime=fProton.DeltaTime();
  
  TD.KpP=0;
  TD.KpTh=0;
  TD.KpTime=0;
  
  TD.KmP=fKm.P4p()->P();
  TD.KmTh=fKm.P4p()->Theta()*TMath::RadToDeg();
  TD.KmTime=fKm.DeltaTime();
  
  TD.BeamTime=fBeam.DeltaTime();

}
void K2::Topo_3(){
  //For topology Beam:Kp:Km

  //Subtract starttime from all particles
  //This has been predetermined from the tagger time
  fTrigger.SubtractStartTime(&fKp,&fKm);
  fTrigger.SubtractStartTimeBeam(&fKp,&fBeam);

  fKp.TakeCorrectedP();//Apply eloss
  fKm.TakeCorrectedP();//Apply eloss

//Reconstruct missing or combined particles
  HSLorentzVector miss= fBeam.P4() + fTarget -fKp.P4() -fKm.P4();
  TD.MissMass2=miss.M2();
  TD.MissMass=miss.M();
  
  fProton.FixP4(miss);

  //particle kine
  TD.ProtP=0;
  TD.ProtTh=0;
  TD.ProtTime=0;
  
  TD.KpP=fKp.P4p()->P();
  TD.KpTh=fKp.P4p()->Theta()*TMath::RadToDeg();
  TD.KpTime=fKp.DeltaTime();
  
  TD.KmP=fKm.P4p()->P();
  TD.KmTh=fKm.P4p()->Theta()*TMath::RadToDeg();
  TD.KmTime=fKm.DeltaTime();
  
  TD.BeamTime=fBeam.DeltaTime();

}

Created file '/scratch/dglazier/testg8/HASPECT6/hsexamples/ToposK2.h'.


## Common Calculations : Kinematics
The final state kinematics are common to all topologies so we only need to write one function.
We can also make use of the Kinematics calculator HSKinematics (use as fKine)  for calculating standard production and decay quantities

%edit KinematicsK2.h

In [3]:
%%file KinematicsK2.h
#include "K2.h"
void K2::Kinematics(){

  //Calculate possible resonances
  HSLorentzVector pMeson=fKp.P4()+fKm.P4();
  HSLorentzVector pHyperon=fProton.P4()+fKm.P4();
  //invariant masses
  TD.MesonMass=pMeson.M();
  TD.HyperonMass=pHyperon.M();
  
  fKine.SetGammaTarget(fBeam.P4(),fTarget);
  TD.Eg=fBeam.P4p()->E(); //photon bem energy

   TD.PolState=0;
   TD.Pol=fBeam.Vertex().X();
   if(TD.Pol)
      TD.PolState=1;
    else{
      TD.Pol=fBeam.Vertex().Y();
      if(TD.Pol)TD.PolState=-1;
    }
    
  //calculate CM production kinematics for meson
  fKine.SetMesonBaryon(pMeson,fProton.P4());
  fKine.PhotoCMDecay();
  TD.t=fKine.t();
  TD.MesonCosTh=fKine.CosTheta();//prefix all variables to be saved wiht TM.
  TD.MesonPhi=fKine.Phi();
  
  //Get phi decay variables
  fKine.SetMesonDecay(fKp.P4(),fKm.P4());
  fKine.MesonDecayHelicity();
  TD.HelCosTh=fKine.CosTheta();
  TD.HelPhi=fKine.Phi();

  //calculate CM production kinematics for hyperon
  fKine.SetMesonBaryon(fKp.P4(),pHyperon);
  fKine.PhotoCMDecay();
  TD.HyperonCosTh=fKine.CosTheta();//prefix all variables to be saved wiht TM.
  TD.HyperonPhi=fKine.Phi();
  

}

Created file '/scratch/dglazier/testg8/HASPECT6/hsexamples/KinematicsK2.h'.


## Tree Data
The data you write in a tree is connected via a TreeData class which has data members which will be the tree leafs i.e. data you can perform further analysis on (e.g. histogram, event fitting) 

%edit TreeDataK2.h

In [4]:
%%file TreeDataK2.h
#ifndef TREEDATAK2_h
#define TREEDATAK2_h 
#include "TreeData.h"

class TreeDataK2: public HS::TreeData{
 public:
  //data member for tree branches below here
  Double_t MissMass=0;
  Double_t MissMass2=0;
  
  Double_t MesonMass=0;
  Double_t HyperonMass=0;
  
  Double_t MesonCosTh=0;
  Double_t MesonPhi=0;
  Double_t HyperonCosTh=0;
  Double_t HyperonPhi=0;
  Double_t Eg=0;
  Double_t t=0;
  
  Double_t HelCosTh=0;
  Double_t HelPhi=0;
  
  Double_t ProtP=0;
  Double_t ProtTh=0;
  Double_t ProtTime=0;
  Double_t KpP=0;
  Double_t KpTh=0;
  Double_t KpTime=0;
  Double_t KmP=0;
  Double_t KmTh=0;
  Double_t KmTime=0;
  Double_t BeamTime=0;

  Double_t Pol=0;
  Int_t PolState=0;
  
  ClassDef(TreeDataK2,1);
};
#endif

Created file '/scratch/dglazier/testg8/HASPECT6/hsexamples/TreeDataK2.h'.


Now you can try and compile all of the code together. You may need to go back and fix bugs after. This is best done by removing the %edit cells and doing a restart and run whole notebook.

In [5]:
gROOT->ProcessLine(".x LoadK2.C+");

%%%%%%%%%%%%%%%%%%%%%%%%%    THSParticle
%%%%%%%%%%%%%%%%%%%%%%%%%    EventInfo
%%%%%%%%%%%%%%%%%%%%%%%%%    RunInfo


In file included from LoadFinalState_C_ACLiC_dict dictionary payload:10:
/scratch/dglazier/tesths/HASPECT6/hsfinalstate/LoadFinalState.C:11:6: error: redefinition of 'LoadFinalState'
void LoadFinalState(){
     ^
/scratch/dglazier/testg8/HASPECT6/hsfinalstate/LoadFinalState.C:11:6: note: previous definition is here
void LoadFinalState(){
     ^
Warning in <TInterpreter::TCling::RegisterModule>: Problems declaring payload for module LoadFinalState_C_ACLiC_dict.


%%%%%%%%%%%%%%%%%%%%%%%%%    THSWeights
%%%%%%%%%%%%%%%%%%%%%%%%%    DataManager
%%%%%%%%%%%%%%%%%%%%%%%%%    LundReader
&&&&&&&&&&&&&&&&&&&&&&&&&&&& HSKinematics
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Cuts
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Combitorial
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleIter
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Topology
&&&&&&&&&&&&&&&&&&&&&&&&&&&& FinalState
&&&&&&&&&&&&&&&&&&&&&&&&&&&& FiledTree
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreeData
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TopoActionManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleCuts
&&&&&&&&&&&&&&&&&&&&&&&&&&&& VarsParticle
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreePrepParticle
&&&&&&&&&&&&&&&&&&&&&&&&&&&& MVASignalID
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleCutsManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreePrepManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& MVASignalIDManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& CLAS/CLASTrigger
&&&&&&&&&&&&&&&&&&&&&&&&&&&& CLAS12/CLAS12Trigger
&&&&&&&&&&&&&&&&&&&&&&&&&&&& CLAS12/CLAS12DeltaTime


Info in <TUnixSystem::ACLiC>: creating shared library /scratch/dglazier/testg8/HASPECT6/hsexamples/./TreeDataK2_C.so
Info in <TUnixSystem::ACLiC>: creating shared library /scratch/dglazier/testg8/HASPECT6/hsexamples/./K2_C.so


In [6]:
K2 kk;

 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
 WARNING running HS::FinalState::AutoIter() you  should check the topology print output and make sure the displayed iterator is waht you want ..
 Topology::Print() : 0
    particles = -10000 0 10000 10000 
You can have any number of the following particles : 
 ALL 
The following particle are identified by pdg code : 
 ALL 
 ParticleIter::Print() 
     Type : 2 number chosen 1 of id  -321 and number used here = 1
 ParticleIter::Print() 
     Type : 2 number chosen 1 of id  -22 and number used here = 1
 ParticleIter::Print() 
     Type : 2 number chosen 1 of id